In [1]:
import requests
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns
sns.set(style="whitegrid")
sns.set_context("notebook", font_scale=1.5)
%matplotlib inline

In [2]:
def count_by(df,group_vars,counter_col,count_col_label='total'):
    df_count = df.groupby(group_vars, as_index=False)[counter_col].count() 
    df_count = df_count.rename(columns={counter_col:count_col_label})
    df_count = df_count.sort_values(by=count_col_label, ascending=False,ignore_index=True)
    return df_count

In [6]:
all_releases_df = pd.read_csv('../all_releases_db_comm_caja_16092020.csv')
all_releases_df['Tipo'] = 'SAE'
all_releases_df.loc[(all_releases_df['master_year']>1995) | (all_releases_df['release_year']>1995) | (all_releases_df['format_type']=='CD'),'Tipo'] = 'no SAE'
all_releases_df.loc[all_releases_df['master_year']==0,'Tipo'] = 'desconocido'
all_releases_df

,uid,artist,album,genre,style,style2,format,format_type,release_year,master_year,decade,label,release_type,reissue,release_dif_time,Tipo
0,Dii_box,AIR,"10,000 Hz Legend",Electronic,Downtempo,Electro,Vinyl,LP,2001,2001.0,00's,Source,Album,0,0.0,no SAE
1,Dii_box,AIR,Talkie Walkie,Electronic,Downtempo,Synth-pop,Vinyl,LP,2004,2004.0,00's,Source,Album,0,0.0,no SAE
2,Dii_box,Apparat,LP5,Electronic,Ambient,NaN,Vinyl,LP,2019,2019.0,10's,Mute,Album,0,0.0,no SAE
3,Dii_box,Apparat,Walls,Electronic,Leftfield,NaN,Vinyl,LP,2016,2007.0,00's,Shitkatapult,Album,1,9.0,no SAE
4,Dii_box,Bauhaus,Bela Lugosi's Dead,Rock,New Wave,NaN,Vinyl,"12""",1979,1979.0,70's,Small Wonder Records,Single,0,0.0,SAE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12808,rotblot,Wire,Chairs Missing,Rock,Post-Punk,NaN,Vinyl,LP,0,1978.0,70's,Harvest,Album,1,-1978.0,SAE
12809,rotblot,Wire,Pink Flag,Rock,Punk,NaN,Vinyl,LP,1978,1977.0,70's,Harvest,Album,1,1.0,SAE
12810,rotblot,Wire,Pink Flag,Rock,Punk,NaN,CD,CD,0,1977.0,70's,Parlophone,Album,1,-1977.0,no SAE
12811,rotblot,The XX,I See You,Electronic,Indie Pop,NaN,Box Set,Box Set,2017,2017.0,10's,Young Turks,Deluxe Edition,0,0.0,no SAE


# Estadísticas sobre las colecciones de la comunidad de la [cajasoundbox](https://lacajasoundbox.com)
**Por Gabriel Castrillon (@gabocas en Twitter)***

Una actividad cotidiana en esta comunidad es mostrar lo que rueda en nuestros tocadiscos, dándonos la oportunidad de compartir nuestras colecciones, aprender sobre diferentes versiones de ese disco que tanto nos gusta o añadir nuevos discos a nuestra lista de deseos en Discogs. Adicionalmente, ha generado discusiones sobre cuántos discos podemos reunir entre todos nosotros, cuáles de estos son [SAE](https://lacajasoundbox.com/blogs/comunidad/glosario-esencial-para-los-cajiamigos-coleccionistas) o la discusión fija semanal sobre cuál es el mejor álbum de Soda Stereo. 

Así que para conocer mejor nuestras colecciones, esculcamos 26 de ellas en Discogs, para un total de 12813 vinilos, cassetes, DVDs, etc, a los cuales me referiré de ahora en adelante como discos. El formato más coleccionados es el vinilo (75%, 71% SAE), seguido por el CD (20%), DVD (2%) y cassette (1.5%). De nuestro formato ganador, la mayoría de los discos coleccionados fueron lanzados en la década de los 80's (33%, 89% SAE), seguida por los 70's (27%, 92% SAE) y los 90's (33.3%, 49% SAE).


In [66]:
cm_466 = ['#005867','#FFB335','#F3F1EA','#2E383D'] #taken from https://www.instagram.com/colours.cafe/
releases_by_format_decade = count_by(all_releases_df,['format','decade','Tipo'],'album')
px.sunburst(releases_by_format_decade, path=['format','decade','Tipo'], values='total',color_discrete_sequence=cm_466)


En general, los géneros más coleccionados son el Rock (62%)🤘, específicamente Rock Alternativo (13%), Pop Rock (12%), Hard Rock (11%) y Heavy Metal (5%); seguido por Electrónica (19%), específicamente Synth-pop (18%), New wave (9%) y Alternativo (9%); y Latin (6%), específicamente Salsa (45%), Cumbia (9%) y Bolero (6%). Cabe aclarar que los diferentes géneros y subgéneros usado en esta gráfica fueron extraídos directamente de la base de datos de Discogs.

In [18]:
releases_by_genre_style = count_by(all_releases_df,['genre','style'],'album')
px.sunburst(releases_by_genre_style, path=['genre','style'], values='total',color_discrete_sequence=cm_466)

In [62]:
count_label='label'
n = 26 #Colecciones esculcadas
M = all_releases_df.shape[0]
offset_top=0
n_top = 100
sns.set(style="whitegrid")
sns.set_context("notebook", font_scale=1.3)
# Por album/artista
all_releases_unique_df = all_releases_df.groupby(['uid','artist','album','genre','format','format_type'], as_index=False)[count_label].agg('count')
all_releases_unique_df = all_releases_unique_df.rename(columns={count_label:'count'})
print(all_releases_unique_df['count'].sum())
# Por artista
all_releases_artist_stats = all_releases_unique_df.groupby(['artist'], as_index=False)['count'].sum() #,'decade'
all_releases_artist_stats = all_releases_artist_stats.rename(columns={'count':'total'})
all_releases_artist_stats = all_releases_artist_stats[all_releases_artist_stats.artist != 'Various']
all_releases_artist_stats = all_releases_artist_stats.sort_values(by='total', ascending=False,ignore_index=True)
all_releases_artist_stats['artist'] =  (all_releases_artist_stats.index+1).astype(str).str.zfill(3) + ' - ' + all_releases_artist_stats['artist']
# Por artista/album/id
all_releases_album_stats = all_releases_unique_df.groupby(['artist','album'], as_index=False)['uid'].count()#,'decade'
all_releases_album_stats = all_releases_album_stats.rename(columns={'uid':'únicos'})
all_releases_album_sum = all_releases_unique_df.groupby(['artist','album'], as_index=False)['count'].sum()#,'decade'
all_releases_album_stats['total'] = all_releases_album_sum['count']
all_releases_album_stats = all_releases_album_stats.sort_values(by='total', ascending=False,ignore_index=True)
all_releases_album_stats['artist_album'] =  (all_releases_album_stats.index+1).astype(str).str.zfill(3) + ' - ' + all_releases_album_stats['artist'] + ' - ' + all_releases_album_stats['album']


12813


Los artistas más coleccionados son Pink Floyd (2.1%), Metallica (2%), David Bowie (1.8%), Queen (1.7%) y nuestro polémico Soda Stereo (0.9%). Un dato interesante de nuestro Top 5 de Artistas es el gran impacto que tienen las colecciones de los ultra-fans en estas estadísticas (outliers), ya que solo una colección de distintos amigos de la comunidad puede contribuir con el 68% de los discos de Metallica, 50% de Queen, 39% de Bowie, 34% de Soda Stereo y 30% de Pink Floyd.

In [65]:
px.bar(all_releases_artist_stats.sort_values(by='total', ascending=False)[offset_top:offset_top+n_top], 
       x="total", y="artist", orientation='h',color_discrete_sequence=cm_466,template='plotly_white',
       height=1100).update_xaxes(title = 'Número de discos').update_yaxes(nticks=n_top,
                                                                          tickmode='linear',
                                                                          tickfont={'size':9},
                                                                          title = 'Artista',
                                                                          categoryorder="category descending")


Para terminar, los álbumes más coleccionados son The Dark Side of the Moon (0.24%), Master of Puppets (0.19%), Ride the Lightning (0.16%), The Wall (0.16%) y Kill'em All (0.16%). Un aspecto interesante que puede apreciarse en la gráfica, es que la muchos de los discos son coleccionados más de una vez por la misma persona (discos repetidos en amarillo). Espero que les haya gustado esta entrada, así que no duden en compartir sus comentarios o ideas para futuras publicaciones en este tema, pues la información que podemos extraer de Discogs es gigante y personalmente tengo muchas ideas sobre otras estadísticas bien interesantes que se podrían generar con nuestras colecciones. Muchas gracias a todos los amigos de la caja por compartir sus discogs ids y a Juan Rafael por su ayuda en la selección del punto de corte para clasificar un disco como [SAE](https://lacajasoundbox.com/blogs/comunidad/glosario-esencial-para-los-cajiamigos-coleccionistas): 1995. Como bonus track y aportando con la discusión semanal de la comunidad, les dejo el Top de álbumes más coleccionados de Soda Stereo: Canción Animal (14%), Doble Vida (12%), Signos (10%), Soda Stereo (10%) y un empate técnico entre el Sueño Stereo y el Dynamo con el 7.38% (en serio!).

In [63]:
all_releases_album_stats_melt = all_releases_album_stats.copy()
all_releases_album_stats_melt['repetidos'] = all_releases_album_stats_melt['total'] - all_releases_album_stats_melt['únicos']
all_releases_album_stats_melt = pd.melt(all_releases_album_stats_melt[offset_top:offset_top+n_top],id_vars=['artist_album'], 
                                        value_vars=['únicos','repetidos'],var_name='Tipo')
px.bar(all_releases_album_stats_melt, x="value", y="artist_album", orientation='h',color='Tipo',
       color_discrete_sequence=cm_466,barmode='stack',template='plotly_white',
       height=1100).update_xaxes(title = 'Número de discos').update_yaxes(nticks=n_top,
                                                                         tickmode='linear',
                                                                         tickfont={'size':9},
                                                                         title = 'Artista - Álbum',
                                                                         categoryorder="category descending")